# Étude de cas

# Préambule

<span style="background: red; color: white;">**!!! Avant de commencer, exécutez le contenue des cellules suivantes.**</span>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (SparkSession.builder
    .appName("store-stock")
    .master("local[*]")
    .config("spark.ui.showConsoleProgress", "True")
    .config("spark.executor.memory", "2g")
    .getOrCreate())

# Access the JVM and import the required Java classes
jvm = spark.sparkContext._jvm
Level = jvm.org.apache.logging.log4j.Level
Configurator = jvm.org.apache.logging.log4j.core.config.Configurator

# Set the root level to OFF
Configurator.setRootLevel(Level.OFF)

In [ ]:
# Display an extract of a dataset and its schema.
def display(dataset):
    dataset.show(10, True)
    dataset.printSchema()

# Introduction

Ce notebook est divisé en 4 parties, plus une introduction :

 * Introduction
 * I -   Données de stock
 * II -  Données de commande client
 * III - Projection de stock
 * IV -  Application d'un seuil commandable
 
Les données nécessaires pour ce notebook sont dans le répertoire `data/`.

Toutes les questions sont surlignées en <span style="background: yellow;">jaune</span>.

Durant l'évaluation, vous avez un accès à l'ensemble des supports, incluant ceux se trouvant sur Internet. Par contre, la communication entre étudiants n'est pas permise.

 * API Python de Spark SQL : https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html
 
<span style="background: #ffbbaa;">**Pensez à sauvegarder régulièrement votre notebook.**</span>

## Commandor: site e-commerce

Nous considérons ici _Commandor_, une entreprise de grande distribution, possédant un ensemble de magasins de type hypermarché et supermarché, ainsi que des magasins de proximité. Ces magasins proposent à ses clients un ensemble de produits achetables sur site.

Ces produits sont pour la plupart proposés aussi sur le site e-commerce de l'entreprise. Une fois les produits sélectionnés, les clients ont la possibilité de choisir le jour de livraison. Un client peut proposer une date de livraison pouvant aller jusqu'à 7 jours.

### Besoin
Commandor a besoin d'un traitement fournissant pour son site e-commerce des projections de stocks. Ces projections doivent permettre aux utilisateurs du site de se faire une idée assez précise du stock disponible pour chacun des produits qu'il commande. Les projections permettent aussi de déterminer les éventuels produits manquants, les besoins en termes de produits à commander auprès des fournisseurs...

### Identifiants usuels dans les données
Sur chaque source de données, nous retrouvons 4 identifiants. Ces identifiants doivent aussi se retrouver dans le résultat des traitements.

#### Store
Le parc de Commandor est composé de magasins ayant chacun un identifiant unique que l'on retrouve dans le champ `store` des données. Le parc Commandor contient environ 1000 magasins sur le territoire national.

#### Service
Chaque magasin est décomposé en un ou plusieurs services. En général, ces services correspondent au magasin lui-même, mais il peut y avoir d'autres services comme un Drive, par exemple. Par contre, un service est rattaché à un unique magasin. Chaque service possède un identifiant unique que l'on retrouve dans le champ `service` des données. Il y a en moyenne 1,25 services par magasin.

#### Product / Barcode
Chaque produit possède deux références uniques au niveau national : un identifiant interne appelé `product` et un code barre appelé `barcode`. Commandor référence près d'un million de produits en incluant aussi bien les produits commandables sur le site e-commerce, que les produits achetables uniquement en physique.

# I - Stock

## Inventaires de stock
Chaque magasin remonte par voie électronique des inventaires de stock. Ces inventaires indiquent pour chaque magasin/service et pour chaque produit la quantité constaté. Un inventiare est forcément accompagné d'un horodatage (ie. date et heure du constat du niveau de stock pour le produit et le service considéré).

En journée, la plateforme de données reçoit des inventaires partiels ; c'est-à-dire, des inventaires uniquement sur des produits pour lesquels des mouvements ont été constatés. En fin de journée, chaque magasin envoit un inventaire complet de son stock.

## À faire
<span style="background: yellow;">Récupérer le contenu des fichiers contenus dans le répertoire `data/stocks/`.</span> Mettez les données en cache.

In [ ]:
stocks = spark.read.parquet("data/stocks")

<span style="background: yellow;">Afficher un extrait des données de stocks et afficher aussi le schéma de ces données.</span>

In [ ]:
display(stocks)

# II -  Orders
## Commandes client
Un flux de données est mis en place contenant les données des commandes client. Une commande client contient un identifiant unique de commande, le magasin et le service où est effectué la commande, la date de création de la commande, la date prévisionnelle de livraison, la liste des statuts passés et actuels de la commande et la liste des produits achetés, accompagnés de leur quantité respective.

Le statut d'une commande varie au cours du temps. Initialement, la commande est `CAPTURED`, puis elle va passer par différents statuts, comme `AFFECTED` ou `IN_PREPARATION`, avant de passer au statut `DELIVERED`. Un client peut éventuellement annuler une commande. Si l'annulation est acceptée, le commande passe au statut `CANCELLED`.

Les commandes stockent dans une liste l'ensemble des statuts par lesquels elle est passée jusque-là et la date à laquelle la commande est passée par ces différents statuts.

## À faire
<span style="background: yellow;">Récupérez le contenu des fichiers contenus dans le répertoire `data/orders/`.</span>

In [ ]:
orders = spark.read.parquet("data/orders")

<span style="background: yellow;">Afficher un extrait des données de commandes client et afficher aussi le schéma de ces données.</span>

In [ ]:
display(orders)

## Nettoyage
Les données de commandes contiennent des lignes qui ne rentrent pas le périmètre à prendre en compte.

<span style="background: yellow;">Retirez des données les commandes dont la date de livraison est antérieur au _24 janvier 2022_.</span>

In [ ]:
clean_orders = ???

display(clean_orders)

## Priorisation des statuts
Dans ces commandes, nous avons l'ensemble des statuts par lesquels elles sont passées. Seul le dernier statut (ie. le statut le plus récent) nous intéresse, excluant les statuts ayant une priorité égale ou supérieure à _livrées_ (`DELIVERED`).

Le code ci-dessous est fourni pour gérer les priorités des statuts. <span style="background: yellow;">Exécutez-le et passez à la cellule suivante.</span>

In [ ]:
class Status:
  Captured = "CAPTURED"
  Categorized = "CATEGORIZED"
  Affected = "AFFECTED"
  Changed = "CHANGED"
  Validated = "VALIDATED"
  Optimized = "OPTIMIZED"
  Prioritized = "PRIORITIZED"
  Routed = "ROUTED"
  CancellationKo = "CANCELLATION_KO"
  InPreparation = "IN_PREPARATION"
  Prepared = "PREPARED"
  Received = "RECEIVED"
  Delivered = "DELIVERED"
  Cancelled = "CANCELLED"
  Invoiced = "INVOICED"
  Expediable = "EXPEDIABLE"

  orderedStatuses = [
      Captured,
      Categorized,
      Affected,
      Changed,
      Validated,
      Optimized,
      Prioritized,
      Routed,
      CancellationKo,
      InPreparation,
      Prepared,
      Received,
      Delivered,
      Cancelled,
      Invoiced,
      Expediable
  ]

  statusesByPriority = {value: index for index, value in enumerate(orderedStatuses)}

  # Give the priority index of a status.
  # It starts from 0 (lowest) to 15 (higher). If
  # the status is unknown, the priority returned
  # is -1.
  def priorityOf(status):
    return Status.statusesByPriority.get(status, -1)

À partir du code ci-dessus, <span style="background: yellow;">créez une UDF qui à partir d'une colonne contenant un statut donne sa priorité.</span>

In [ ]:
priorityOfStatus = ???

## Nettoyage selon les statuts (/4)
Maintenant, <span style="background: yellow;">retirez toutes les commandes déjà livrées (ie. ayant un statut de priorité inférieur à `DELIVERED`)</span>, pour n'avoir que les commandes en cours.

In [ ]:
# Convert the dataset to have one line per status
orders_with_status = ???

# Associate to each order its status priority
orders_with_status_priority = ???

# Create a dataset that for each order ID get the highest status priority
last_status_priority = ???

# Transform the dataset to only keep the pending orders (ie. orders not delivered)
pending_orders = ???

display(pending_orders)

## Produits commandés
Afin de pouvoir rapprocher les commandes client avec les inventaires de stock, il faut réorganiser les données de commande. Nous avons actuellement une ligne de données par commande, sachant qu'une commande contient une liste de produits commandés. Il nous faut maintenant une ligne de données par produit commandé.

<span style="background: yellow;">Transformez les données de commande client afin d'avoir une ligne par produit commandé avec leur quantité respective.</span> Attention, il est possible qu'il y ait des doublons. Pensez à les retirer. Mettez aussi les données en cache.

In [ ]:
orders_by_item = ???

display(orders_by_item)

# III - Projections simples
Une projection consiste à simplement prendre les données de stocks et à en retirer les quantités commandées.

## Quantités achetées
<span style="background: yellow;">Convertissez les données de produits commandés afin d'avoir la quantité totale commandées par produit.</span>

In [ ]:
bought_quantities = ???

display(bought_quantities)

## Stock et commande
<span style="background: yellow;">Associez les données de stock avec l'ensemble des produits commandés.</span>

In [ ]:
stock_and_order = ???

display(stock_and_order)

## Projection
Une projection fournit des informations sur le stock à venir, une fois les commandes clients, actuellement en cours, satisfaites.

<span style="background: yellow;">Convertissez les données calculées précédemment pour obtenir les projections de stock.</span> Mettez les données en cache.

Les données devront avoir uniquement les colonnes suivantes :

 * identifiant du produit (`product`)
 * code barre du produit (`barcode`)
 * identifiant du magasin (`store`)
 * identifiant du service (`service`)
 * horodatage de création de la projection (`projection_timestamp`)
 * quantité actuelle dans le stock (`stock_quantity`)
 * quantité projetée (= quantité dans le stock - quantité commandée) (`projected_quantity`)
 * disponibilité du produit dans le service (`true` = produit disponible (stock projeté > 0) ; `false` = produit indisponible (stock projeté <= 0)) (`available`)

In [ ]:
projections = ???

display(projections)

# IV - Application d'un seuil
Afin de donner une certaine priorité aux clients venant sur site (ie. directement en magasin), une partie du stock leur est réservé. Cela se fait en appliquant un seuil en dessous duquel il n'est pas possible de commander des produits sur le site e-commerce.

Le métier propose de fixer le seuil à 5. À l'exception de certains produits dans certains magasins pour lesquels le seuil est à 8. Les listes des magasins et des produits associés auxquels s'applique l'exception sont données dans le code ci-dessous.

Par exemple
 * Magasin `9999999999999` (qui n'est pas dans la liste), produit `9999999` (qui n'est pas dans la liste): en quantité en stock = 15, stock commandable = 10.
 * Magasin `3021080366104` (dans la liste), produit `9999999` (qui n'est pas dans la liste): en quantité en stock = 15, stock commandable = 10.
 * Magasin `9999999999999` (qui n'est pas dans la liste), produit `0045325` (dans la liste): en quantité en stock = 15, stock commandable = 10.
 * Magasin `3021080366104` (dans la liste), produit `0045325` (dans la liste): en quantité en stock = 15, stock commandable = 7.
 
<span style="background: yellow;">Exécutez la cellule suivante.</span>

In [ ]:
  storeList = [
    "3021080366104",
    "3020180005883",
    "3021081177693",
    "3021081027905",
    "3021081178997",
    "3021080561301",
    "3020180045766",
    "3021081044704",
    "3021520203433",
    "3021520410619",
    "3021520306097",
    "3021080631301",
    "3021080623108",
    "3021081045701",
    "3021520203341",
    "3021080670409",
    "3021080563800",
    "3021080368306",
    "3021080368405",
    "3021080810102"
  ]

  productList = [
    "0045325",
    "0251136",
    "0549341",
    "2649500",
    "2682819",
    "3332364",
    "3332585",
    "3634957",
    "4027200",
    "4469403",
    "4604802",
    "5154158",
    "5531897",
    "5678019",
    "5717543",
    "5793947",
    "5883984",
    "5903077",
    "5939682",
    "6088956",
    "6126635",
    "6149792",
    "6149950",
    "6163671",
    "6361434",
    "6440091",
    "6596181",
    "6708306",
    "6833035",
    "7033846",
    "7073916",
    "7181476",
    "7185763",
    "7186214",
    "7302277",
    "7314602",
    "0093531",
    "0256105",
    "0691219",
    "1248593"
  ]

<span style="background: yellow;">Corrigez les données de projection afin d'avoir les quantités projetées corrigées du seuil applicable.</span> Pensez à corriger la disponibilité au besoin.

In [ ]:
projections_with_threshold = ???

display(projections_with_threshold)